In [277]:
from utils import Instance,Benchmark
import numpy as np
import itertools as iter

In [1]:
benchmark=Benchmark(20,5,benchmark_folder='../benchmarks')
ben=benchmark.get_instance(0)
ben.np_array

NameError: name 'Benchmark' is not defined

In [ ]:
def get_sequences(instance : Instance):#for what?
    jobs_count = instance.get_jobs_number()
    return list(iter.permutations(range(jobs_count)))
get_sequences(ben)

In [280]:
def generate_initsol(instance: Instance,popul_size=3):#returns an np array containing jobs permutations
  #initial array
  nb_jobs=instance.get_jobs_number()
  arr=np.array([])
  res=[]
  arr=[int(i) for i in range(nb_jobs)]
  for i in range(popul_size):
    res.append(np.random.permutation(arr))
  res=np.array(res)
  res=np.unique(res,axis=0)
  return res,len(res)
init_sol,pop= generate_initsol(ben,10)
#init_sol=np.unique(init_sol,axis=0)
init_sol

array([[0, 1, 4, 2, 3],
       [0, 2, 1, 3, 4],
       [0, 2, 3, 4, 1],
       [1, 3, 2, 4, 0],
       [2, 0, 1, 4, 3],
       [2, 4, 0, 1, 3],
       [3, 2, 1, 4, 0],
       [4, 1, 3, 2, 0],
       [4, 3, 0, 1, 2]])

In [281]:
def Cmax(instance: Instance, chromosome):
  n=instance.get_jobs_number()
  m=instance.get_machines_number()
  span=np.zeros(m)
  for i in range(n):
    for j in range(m):
      if (j==1):
        span[j]+=ben.np_array[chromosome[i]][j]
      else:
        span[j]=max(span[j-1],span[j])+ ben.np_array[chromosome[i]][j]
  return span[m-1]

Cmax(ben,init_sol[0])


535.0

In [282]:
#Fitness function: un individu a une forte probabilité de se reproduire si il a une bonne qualité (c-à-d une grande valeur de fitness)
def fitness(instance: Instance, chromosome):
  return 1/Cmax(instance,chromosome)
fits=np.array([])
for i in range(len(init_sol)):
  fits=np.append(fits,fitness(ben,init_sol[i]))
fits

array([0.00186916, 0.00222717, 0.0018622 , 0.00217391, 0.00194175,
       0.00195695, 0.00184162, 0.00191205, 0.00196078])

In [283]:
#Selection: sélectionne certaines solutions pour former la population
# intermédiaire afin de lui appliquer les opérateurs de croisement et de mutation. 
# Chaque élément de la population est sélectionné avec une probabilité selon sa valeur d’adaptation.
def selection(popul,size,Pc,fits):
  pool_size=round(Pc*size) #définir le nombre d'individus à selectionner pour la reproduction
  fits_copy=fits.copy()
  fits_copy[::-1].sort()
  selected_fits=fits_copy[:pool_size]
  print(selected_fits)
  matting_pool=[]
  i=0
  while i<pool_size:
    index=np.where(fits==selected_fits[i])
    for j in range(len(index[0])):
      matting_pool.append(popul[index[0][j]])
    i+=len(popul[index])
  matting_pool=np.array(matting_pool)
  matting_pool=np.unique(matting_pool,axis=0)
  return matting_pool
matting_pool=selection(init_sol,8,0.9,fits)
matting_pool

[0.00222717 0.00217391 0.00196078 0.00195695 0.00194175 0.00191205
 0.00186916]


array([[0, 1, 4, 2, 3],
       [0, 2, 1, 3, 4],
       [1, 3, 2, 4, 0],
       [2, 0, 1, 4, 3],
       [2, 4, 0, 1, 3],
       [4, 1, 3, 2, 0],
       [4, 3, 0, 1, 2]])

In [284]:
import random
matting_pool=selection(init_sol,8,0.9,fits)
#Crossover: interchanger les gènes situés entres les points considérés et est appliqué avec une probabilité Pc.
#On utilisera ici le croisement à 2 points car des études faites ont constaté que ce type de croisement est très efficace pour le FSP
def crossover(n,matting_pool,Pc):
  offsprings=[]
  nb_cross=round(Pc*len(matting_pool))
  for i in range (nb_cross):
    #generate randomly 2 crossover points
    points=np.random.randint(n-1,size=2)
    first_pt=points[0]
    second_pt=points[1]
    
    while True:
      if (first_pt != second_pt): 
        break
      second_pt=random.randint(0,n-1)

    if (first_pt>second_pt):
        t=first_pt
        first_pt=second_pt
        second_pt=t
    
    #choose randomly 2 parents
    rand_ind=np.random.choice(len(matting_pool), size=2)
    parents=matting_pool[rand_ind]
    # parents = np.random.choice(matting_pool, size=2, replace=False)
    parent1=parents[0]#[0]
    parent2=parents[1]#[0]    

    offspring=parent1
    parts_parent1=[]
    parts_parent1[0:first_pt]=parent1[0:first_pt]
    if (first_pt!=0):
      parts_parent1[first_pt+1:first_pt+(n-second_pt)+1]=parent1[second_pt:n]
    else:
      parts_parent1[first_pt:first_pt+(n-second_pt)+1]=parent1[second_pt:n]
    #print(parts_parent1)
    idx=first_pt
    for i in parent2:
      if (i not in parts_parent1):
        offspring[idx]=i
        idx+=1
    offsprings.append(offspring)
  return offsprings

crossover(5,matting_pool,0.5)

[0.00222717 0.00217391 0.00196078 0.00195695 0.00194175 0.00191205
 0.00186916]


[array([2, 0, 1, 4, 3]),
 array([4, 3, 0, 1, 2]),
 array([4, 3, 0, 1, 2]),
 array([4, 1, 3, 2, 0])]

In [285]:
#Mutation: consiste à choisir un ou deux bits aléatoirement, puis les inverser. L'opérateur de mutation s'applique avec une certaine probabilité Pm, appelée
#taux de mutation
# Dans le cas d'une mutation par changement de poste, une tâche située à un poste est supprimée et placée à un autre poste. Ensuite, tous les autres postes 
#sont déplacés en conséquence. Les deux positions sont choisies au hasard.
offsprings=crossover(5,matting_pool,0.5)
def mutation(n,offsprings,Pm):
  for offspring in offsprings:
    pos = np.random.choice(n, size=2, replace=False)
      
    if pos[0] > pos[1]:
        t = pos[0]
        pos[0] = pos[1]
        pos[1] = t
    
    remJob = offspring[pos[1]]
    
    for i in range(pos[1], pos[0], -1):
        offspring[i] = offspring[i-1]
        
    offspring[pos[0]] = remJob
  offsprings=np.array(offsprings)
  return offsprings
mutation(5,offsprings,1)

array([[0, 2, 1, 4, 3],
       [0, 1, 3, 2, 4],
       [1, 2, 3, 4, 0],
       [1, 0, 2, 3, 4]])

In [286]:
#new_generation: la nouvelle génération est constituée des meilleurs individus de la population
offsprings=mutation(5,offsprings,1)
init_sol=np.concatenate((init_sol,offsprings),axis=0)
#print("\n",init_sol,len(init_sol))
fits=np.array([])
for i in range(len(init_sol)):
  fits=np.append(fits,fitness(ben,init_sol[i]))#fits déja calculés

def new_generation(n,init_sol,popul_size,fits):
  new_gen=np.array([])
  new_gen=selection(init_sol,popul_size,1,fits)
  if (len(new_gen)<popul_size):
    adds=popul_size-len(new_gen)
    i=0
    k=0
    print(new_gen)
    while i<adds:
        for j in init_sol:
          j=np.array(j)
          varr=any((j == x).all() for x in new_gen)
          if (not varr):
            new_gen=np.append(new_gen,j[None,:],axis=0)           
            i+=1
            if (i==adds):
              break
          
  return new_gen
new_generation(5,init_sol,10,fits)

[0.00232019 0.00222717 0.00217391 0.00209644 0.00198807 0.00196078
 0.00195695 0.00194175 0.00191205 0.00188324]


array([[0, 2, 1, 3, 4],
       [0, 3, 1, 2, 4],
       [1, 3, 2, 4, 0],
       [2, 0, 1, 4, 3],
       [2, 1, 0, 3, 4],
       [2, 1, 3, 4, 0],
       [2, 4, 0, 1, 3],
       [4, 0, 2, 1, 3],
       [4, 1, 3, 2, 0],
       [4, 3, 0, 1, 2]])

In [287]:
import random,time,json
def get_results(instance=ben,popul_size=12,nb_generations=50,Pc=0.9,Pm=0.06):
  j=0
  n=instance.get_jobs_number()
  start=time.time()
  while j<nb_generations:

    #Generate an inital population
    init_pop,size=generate_initsol(instance,popul_size)
    print("old population",init_pop)

    #Reproduction: pick the best chromosomes (tournament selection)
    #evaluation
    fits=np.array([])
    for i in range(len(init_pop)):
      fits=np.append(fits,fitness(ben,init_pop[i]))#fits déja calculés
    #print("fits",fits)
    #reproduction
    print(size)
    matting_pool=selection(init_pop,size,Pc,fits)
    print("matting_pool",matting_pool)

    #Crossover: 
    offsprings=crossover(n,matting_pool,Pc)

    #Mutation
    offsprings=mutation(n,offsprings,Pm)
    print("offsprings",offsprings)
    #Selection: 
    init_pop=np.concatenate((init_pop,offsprings),axis=0)
    print("total",init_pop)
    #evaluation
    fits=np.array([])
    for i in range(len(init_pop)):
      fits=np.append(fits,fitness(ben,init_pop[i]))#fits déja calculés
    #print("fits",fits)
    new_gen=new_generation(n,init_pop,popul_size,fits)
    print("new population",new_gen)
    j+=1
  
  #End While, select the best solution from the last generation.
  #choose the best sequence according to the chromosomes' fitnesses => last evaluation
  fits=np.array([])
  for i in range(len(new_gen)):
    fits=np.append(fits,fitness(ben,new_gen[i]))#fits déja calculés
  print("fits",fits)

  fits_sorted=fits.copy()
  fits_sorted[::-1].sort()
  ind=np.where(fits==fits_sorted[0])#find the index of the best solution in the population
  best_sol=new_gen[ind[0][0]]
  end = time.time()
  print(end - start)
  result={
      "best solution":best_sol.tolist(),
      "C_max":Cmax(ben,best_sol),
      "time":end-start
  }

  return json.dumps(result)
get_results()


old population [[0 2 3 1 4]
 [1 0 2 3 4]
 [2 1 0 3 4]
 [2 3 1 4 0]
 [3 1 4 0 2]
 [3 4 0 1 2]
 [4 0 1 3 2]
 [4 1 0 2 3]
 [4 3 0 2 1]]
9
[0.00238095 0.00232019 0.00215054 0.0019802  0.00196078 0.00194553
 0.00192308 0.00191571]
matting_pool [[0 2 3 1 4]
 [1 0 2 3 4]
 [2 1 0 3 4]
 [2 3 1 4 0]
 [3 1 4 0 2]
 [3 4 0 1 2]
 [4 0 1 3 2]
 [4 3 0 2 1]]
offsprings [[4 2 0 1 3]
 [1 0 2 3 4]
 [0 1 2 3 4]
 [4 2 0 3 1]
 [2 4 0 3 1]
 [2 4 3 0 1]
 [1 0 3 2 4]]
total [[0 2 3 1 4]
 [1 0 2 3 4]
 [2 1 0 3 4]
 [2 3 1 4 0]
 [3 1 4 0 2]
 [3 4 0 1 2]
 [4 0 1 3 2]
 [4 1 0 2 3]
 [4 3 0 2 1]
 [4 2 0 1 3]
 [1 0 2 3 4]
 [0 1 2 3 4]
 [4 2 0 3 1]
 [2 4 0 3 1]
 [2 4 3 0 1]
 [1 0 3 2 4]]
[0.00238095 0.00238095 0.00238095 0.00232019 0.00232019 0.00215054
 0.00199601 0.0019802  0.00196464 0.00196078]
[[0 1 2 3 4]
 [0 2 3 1 4]
 [1 0 2 3 4]
 [1 0 3 2 4]
 [2 1 0 3 4]
 [2 3 1 4 0]
 [2 4 0 3 1]
 [2 4 3 0 1]
 [4 3 0 2 1]]
new population [[0 1 2 3 4]
 [0 2 3 1 4]
 [1 0 2 3 4]
 [1 0 3 2 4]
 [2 1 0 3 4]
 [2 3 1 4 0]
 [2 4 0 3 1]
 